# history
issue_key – уникальный ключ задачи
status – статус задачи
minutes_in_status – количество минут, которое задача находилась в статусе
previous_status – предыдущий статус задачи
started_at – время создания статуса задачи, unix миллисекунды
ended_at – время перехода задачи в другой статус, unix миллисекунды


In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
sqlite3.sqlite_version

'3.37.2'

In [3]:
sqlite3_conn = sqlite3.connect("test.db")
sqlite3_result = pd.read_sql('''SELECT * FROM history''', sqlite3_conn)

print(sqlite3_result)

      issue_key       status  minutes_in_status previous_status  \
0       C-40460         Open              113.0            None   
1       C-40460  In Progress             6289.0            Open   
2       C-40460    In Review             2484.0     In Progress   
3        B-7067       Closed                NaN            Test   
4        B-6168  In Progress              187.0            Open   
...         ...          ...                ...             ...   
11555    E-5945  In Progress             9987.0            Open   
11556    E-5945    In Review            15347.0     In Progress   
11557    E-5945       In Owl                0.0       In Review   
11558    E-5945     Resolved              599.0          In Owl   
11559    E-5945       Closed                NaN        Resolved   

          started_at      ended_at  
0      1669363867000  1.669371e+12  
1      1669370640119  1.669748e+12  
2      1669747964986  1.669897e+12  
3      1668583633246           NaN  
4      165

# SQL1

In [4]:
sqlite3_result["minutes_in_status"].isna().sum()

1378

In [43]:
SQL1_query = '''
SELECT history_sort."group",
round(avg(history_sort.minutes_in_status/60.00), 2) as hours_in_status_open
FROM
(SELECT history.issue_key, SUM(CASE WHEN history.minutes_in_status != 'NaN' THEN history.minutes_in_status ELSE 0 END) as minutes_in_status, SUBSTRING (history.issue_key, 1, 1) as "group"
FROM history
WHERE history.status = 'Open'
GROUP BY history.issue_key
) as history_sort
GROUP BY history_sort."group"
'''

SQL1_result = pd.read_sql(SQL1_query, sqlite3_conn)
SQL1_result

,group,hours_in_status_open
0,A,247.15
1,B,281.51
2,C,595.62
3,D,823.03
4,E,817.94


# SQL2

In [42]:
date_from = pd.Timestamp("2019-01-31 12:00:00")
date_to = pd.Timestamp("2021-12-31 12:00:00")

SQL2_query = f'''
SELECT issue_key, status, datetime(started_at / 1000, 'unixepoch') as time_created
FROM history
WHERE status NOT IN ('Closed', 'Resolved')
  AND time_created >= '{date_from}'
  AND time_created <= '{date_to}'
  AND (ended_at IS NULL OR ended_at = (
      SELECT MAX(ended_at)
      FROM history as h2
      WHERE h2.issue_key = history.issue_key
  ))
GROUP BY issue_key
'''

SQL2_result = pd.read_sql(SQL2_query, sqlite3_conn)
SQL2_result

,issue_key,status,time_created
0,A-28782,Waiting for integration,2020-12-15 11:42:56
1,A-30600,Waiting for integration,2021-02-16 14:49:43
2,A-31053,Open,2020-10-21 15:08:13
3,A-31353,Waiting for integration,2021-03-26 06:48:04
4,A-31573,Waiting for integration,2021-04-15 13:29:39
...,...,...,...
59,D-3730,In Owl,2021-08-31 12:34:12
60,D-3852,In Progress,2021-10-05 12:57:11
61,D-3968,In Owl,2021-10-22 10:57:17
62,D-4352,In Owl,2021-12-21 10:17:39
